-----
-----

In [5]:
# api키로 인가코드 받기
# https://kauth.kakao.com/oauth/authorize?client_id=653cb506f7f83734b59b1946f839f247&redirect_uri=https://naver.com&response_type=code&scope=talk_message,friends

# 친구에게 보내기 (로그인) 사용자토큰 받기
import requests

url = 'https://kauth.kakao.com/oauth/token'
rest_api_key = '653cb506f7f83734b59b1946f839f247' #api키 고정
redirect_uri = 'https://naver.com'
authorize_code = 'MoH5p0E8l0F2i5F4bC5e5JWA59tC9Ijdv6pIBt_B8GetAL-_spbSsHqSGiasvOk-Ta3qtQopyNoAAAF6HQYKGw'
     #인가코드 받은것 작성
    
data = {
    'grant_type':'authorization_code',
    'client_id':rest_api_key,
    'redirect_uri':redirect_uri,
    'code': authorize_code,
    }

response = requests.post(url, data=data)
tokens = response.json()
print(tokens)

{'access_token': 'rAZwAfwgHxzN6q8dB9lT1e3w76koT3q9rLmMbwo9c-wAAAF6HQaH9Q', 'token_type': 'bearer', 'refresh_token': 'lbvGrpIpd1qTueq13Ov1_-66SPK70_tBFHYzUQo9c-wAAAF6HQaH9Q', 'expires_in': 21599, 'scope': 'account_email talk_message profile friends', 'refresh_token_expires_in': 5183999}


## 친구한테 메세지 보내기 토큰 파일

In [6]:
# json 저장
import json
with open("kakao_friend_code.json","w") as fp:
    json.dump(tokens, fp)
    
# 토큰값 읽어오기
with open("kakao_friend_code.json","r") as fp:
    tokens_f = json.load(fp)

In [7]:
# 친구목록 가져오기(uuid 가져오기)
import requests
import json

with open("kakao_friend_code.json","r") as fp:
    tokens_f = json.load(fp)

friend_url = "https://kapi.kakao.com/v1/api/talk/friends"

# GET /v1/api/talk/friends HTTP/1.1
# Host: kapi.kakao.com
# Authorization: Bearer {ACCESS_TOKEN}

headers={"Authorization" : "Bearer " + tokens_f["access_token"]}
result = json.loads(requests.get(friend_url, headers=headers).text)

friends_list = result.get("elements")
friends_id = []

print(requests.get(url, headers=headers).text)
print(friends_list)

for friend in friends_list:
    friends_id.append(str(friend.get("uuid")))

    print(friends_id)

{"error":"invalid_client","error_description":"Bad client credentials","error_code":"KOE010"}
[{'profile_nickname': '민초', 'profile_thumbnail_image': '', 'id': 1769364643, 'uuid': 'qpmvm6uYqp6phbSNuo-3gLSMoJWsm6uSp8A', 'favorite': False}, {'profile_nickname': '세진옵바', 'profile_thumbnail_image': '', 'id': 1770800650, 'uuid': 'qp-rn6eep5ehjbyOuI67jb6MuZWgma6ep5Ln', 'favorite': False}, {'profile_nickname': '주성씨', 'profile_thumbnail_image': 'https://p.kakaocdn.net/th/talkp/wlfmO2BnGD/RjB6BGH2TdqtGIF4F4nGw1/npr5d0_110x110_c.jpg', 'id': 1770794014, 'uuid': 'qp-tnqaRopuphbeEs4Kwh7-OuJShmK-fppP_', 'favorite': False}]
['qpmvm6uYqp6phbSNuo-3gLSMoJWsm6uSp8A']
['qpmvm6uYqp6phbSNuo-3gLSMoJWsm6uSp8A', 'qp-rn6eep5ehjbyOuI67jb6MuZWgma6ep5Ln']
['qpmvm6uYqp6phbSNuo-3gLSMoJWsm6uSp8A', 'qp-rn6eep5ehjbyOuI67jb6MuZWgma6ep5Ln', 'qp-tnqaRopuphbeEs4Kwh7-OuJShmK-fppP_']


In [10]:
# 피트형태 메시지 보내기 (친구에게)
# 기본템플릿(피드형태)

def sendMessage(send_message,send_link):
    header = {"Authorization": 'Bearer ' + tokens_f["access_token"]}
    url = "https://kapi.kakao.com/v1/api/talk/friends/message/default/send" #api 주소
    # uuid = ['qpmvm6uYqp6phbSNuo-3gLSMoJWsm6uSp8A', 'qp-tnqaRopuphbeEs4Kwh7-OuJShmK-fppP_', 'qp-rn6eep5ehjbyOuI67jb6MuZWgma6ep5Ln']
    uuid = ["qpmvm6uYqp6phbSNuo-3gLSMoJWsm6uSp8A"]
    uuidsData = {"receiver_uuids" : json.dumps(uuid)}    

    post = {
        "object_type": "feed",
        "content": {
        "title" : send_message,
        "image_url" : "https://pbs.twimg.com/media/E4Ib3PEUcAg9elB?format=png&name=240x240",
        "image_width" : 200, "image_height" : 200,
        "link" : {
            "web_url" : send_link,
            "mobile_web_url" : send_link         
            },
         },
        "button_title":"공부하러 가기"
    }

    data = {"template_object": json.dumps(post)
           }
    uuidsData.update(data)

    requests.post(url, headers=header, data=uuidsData).status_code
    print(response.status_code)
    if response.json().get('result_code') == 0:
        print('메시지를 성공적으로 보냈습니다.')
    else:
        print('메시지를 성공적으로 보내지 못했습니다. 오류메시지 : ' + str(response.json()))

# 모듈+ API코드 

In [11]:
import tkinter.messagebox as msgbox
from tkinter import *

count_message = 0
count_link = 0

root=Tk()
root.title("카카오톡 메세지 보내기")
root.geometry("360x493")


img=PhotoImage(file="backgroundImg.png", master=root) # 이미지 파일 불러오기
Label(root,image=img, borderwidth=0).place(x=0, y=0)

btn1 = PhotoImage(file="btn1_.png", master=root) #버튼에 이미지 넣기
btn2 = PhotoImage(file="btn2_.png", master=root)

root.resizable(width=False, height=False)

message = Text(root, width=31,height=13,relief="flat", bg="#F0F0F0")

message.insert(END,"메세지를 입력")
link = Entry(root,width=34,relief="flat", bg="#F0F0F0")

link.insert(0,"링크입력")
message.place(x=70, y=82)
link.place(x=58.7, y=308)

def start_message(event):
    global count_message
    if count_message == 0:
        message.delete("1.0",END)
        count_message +=1
        
def start_link(event):
    global count_link
    if count_link == 0:
        link.delete(0,END)
        count_link += 1


def send():
    global send_message
    send_message = message.get("1.0",END)
    print(send_message)
    global send_link
    send_link = link.get()
    print(send_link)
    if(msgbox.askokcancel("메세지 보내기","이대로 보내시겠습니까?")):
        sendMessage(send_message,send_link)
def clear():
    if(msgbox.askokcancel("메세지 초기화","초기화 하시겠습니까?")):
        message.delete("1.0",END)
        link.delete(0,END)

send_message = Button(root,
                      command=send,
                      bg = "snow",
                      fg = "red",
                      relief="flat",
                      borderwidth=0,
                      highlightthickness=0,
                      image=btn1
                     )
clear_message = Button(root,
                       command=clear,
                       bg = "snow",
                       fg = "red",
                       relief="flat",
                       borderwidth=0,
                       highlightthickness=0,
                       image=btn2
                      )
clear_message.place(x=85, y=365)
send_message.place(x=200, y=360)

message.bind("<Button>",start_message)
link.bind("<Button>",start_link)

root.mainloop()

확인

https://japan.hackers.com/?r=japan&c=free_lecture/daily
200
메시지를 성공적으로 보내지 못했습니다. 오류메시지 : {'access_token': 'rAZwAfwgHxzN6q8dB9lT1e3w76koT3q9rLmMbwo9c-wAAAF6HQaH9Q', 'token_type': 'bearer', 'refresh_token': 'lbvGrpIpd1qTueq13Ov1_-66SPK70_tBFHYzUQo9c-wAAAF6HQaH9Q', 'expires_in': 21599, 'scope': 'account_email talk_message profile friends', 'refresh_token_expires_in': 5183999}
아이고 

https://japan.hackers.com/?r=japan&c=free_lecture/daily
200
메시지를 성공적으로 보내지 못했습니다. 오류메시지 : {'access_token': 'rAZwAfwgHxzN6q8dB9lT1e3w76koT3q9rLmMbwo9c-wAAAF6HQaH9Q', 'token_type': 'bearer', 'refresh_token': 'lbvGrpIpd1qTueq13Ov1_-66SPK70_tBFHYzUQo9c-wAAAF6HQaH9Q', 'expires_in': 21599, 'scope': 'account_email talk_message profile friends', 'refresh_token_expires_in': 5183999}
